# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import fowlkes_mallows_score as f

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y.attack_cat.unique())-1, random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = f(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [0]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , fowlkes_mallows_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , fowlkes_mallows_score

(3, 0.9280135423719299)
(9, 0.9056720147456695)
(18, 0.8283396596483319)
(22, 0.7920175580056915)
(12, 0.7875743789460651)
(26, 0.7864076228818787)
(1, 0.784654889170588)
(19, 0.783163915219414)
(4, 0.782785362883871)
(6, 0.782214900190495)
(32, 0.7815590999641648)
(27, 0.7814670453072113)
(23, 0.7809477832911733)
(28, 0.7800995156684721)
(14, 0.7800959020266208)
(15, 0.7800908073922788)
(13, 0.7798133609504193)
(30, 0.778973353242126)
(7, 0.7746310564426178)
(17, 0.7741387422106589)
(16, 0.7720879178261044)
(2, 0.7699643189381294)
(29, 0.7695462619663572)
(0, 0.7393556898883471)
(8, 0.7279000443867022)
(5, 0.7191461814460094)
(33, 0.7080015962025912)
(20, 0.7079990018256839)
(11, 0.7038522033247445)
(21, 0.6967893897632147)
(31, 0.6923536279882361)
(34, 0.6923536279882361)
(35, 0.6923536279882361)
(10, 0.521030523542461)
(40, 0.3546474274464172)
(37, 0.34970300564493595)
(36, 0.348969295947121)
(42, 0.3424669175860176)
(39, 0.3352631644852708)

In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [0]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6667    0.1712    0.2725       619
      Backdoor     0.5976    0.0864    0.1510       567
           DoS     0.3183    0.2731    0.2940      4075
      Exploits     0.6349    0.8058    0.7102     11236
       Fuzzers     0.8960    0.8660    0.8807      6127
       Generic     0.9970    0.9860    0.9915     53886
Reconnaissance     0.9167    0.7445    0.8216      3385
     Shellcode     0.6886    0.6211    0.6531       388
         Worms     1.0000    0.1579    0.2727        38

      accuracy                         0.8905     80321
     macro avg     0.7462    0.5236    0.5608     80321
  weighted avg     0.8939    0.8905    0.8877     80321



In [0]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [0]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.896882829886238 max accuracy
34 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [0]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('service', 0.2408772791820346)
('sbytes', 0.23923046510062784)
('smeansz', 0.1825505752506459)
('Spkts', 0.08104267329591656)
('Sjit', 0.06665930336866442)
('Sintpkt', 0.0627182620904961)
('dbytes', 0.0591994777096565)
('dmeansz', 0.03550493337827944)
('state', 0.03221703062367866)


In [0]:
clf.score(x_train, y_train) 

0.9234007454498767

In [0]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.896882829886238

In [0]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.5896    0.1596    0.2512       495
      Backdoor     0.8491    0.0959    0.1724       469
           DoS     0.5029    0.1082    0.1781      3235
      Exploits     0.6079    0.9204    0.7322      8944
       Fuzzers     0.8942    0.8318    0.8619      4846
       Generic     0.9968    0.9881    0.9924     43219
Reconnaissance     0.9201    0.7470    0.8246      2696
     Shellcode     0.5925    0.4891    0.5358       321
         Worms     0.6538    0.5312    0.5862        32

      accuracy                         0.8969     64257
     macro avg     0.7341    0.5413    0.5705     64257
  weighted avg     0.9004    0.8969    0.8841     64257

